# Deploying functions 

In [75]:
#Reference
#https://docs.openfaas.com/tutorials/kubernetes-hpa/

In [114]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f aeneas.yml --annotation topic="aeneas" --env max_inflight=50 --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=5 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: aeneas.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/aeneas



In [115]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f scaling-aeneas-tocloud.yml --annotation topic="tocloud" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: scaling-aeneas-tocloud.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/scaling-aeneas-tocloud



In [78]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f getobject.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: getobject.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/getobject



In [79]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f pubrabbitmq.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: pubrabbitmq.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/pubrabbitmq



In [80]:
!ssh ubuntu@172.17.141.197 "faas describe scaling-aeneas-mqtt"

No such function: scaling-aeneas-mqtt


# Deploying Rabbitmq Connector 

In [81]:
# Deploying connector configurartion
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f cf.yaml"

configmap/rabbitmq-connector-configmap unchanged


In [120]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply  -f configmap.yaml"

deployment.apps/rabbitmq-connector configured


In [83]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n openfaas" 

NAME                                  READY   STATUS              RESTARTS       AGE
nats-5b77986b8-48p2x                  1/1     Running             4 (6d5h ago)   42d
queue-worker-7d8668444f-tmq7b         1/1     Running             5 (6d5h ago)   28d
alertmanager-6cd7bd5c64-hrcrt         1/1     Running             4 (6d5h ago)   28d
basic-auth-plugin-78b958969c-tnpck    1/1     Running             4 (6d5h ago)   28d
gateway-bf4f5d6f7-4q7kn               2/2     Running             10 (5d ago)    28d
prometheus-7678c644d6-k28wr           1/1     Running             0              9h
rabbitmq-connector-64cbd548f5-szzq2   0/1     ContainerCreating   0              2s


# Deploying KEDA and components 

In [84]:
#Install KEDA 
!ssh ubuntu@172.17.141.197 "sudo kubectl apply -f https://github.com/kedacore/keda/releases/download/v2.9.0/keda-2.9.0.yaml"

namespace/keda unchanged
customresourcedefinition.apiextensions.k8s.io/clustertriggerauthentications.keda.sh unchanged
customresourcedefinition.apiextensions.k8s.io/scaledjobs.keda.sh unchanged
customresourcedefinition.apiextensions.k8s.io/scaledobjects.keda.sh unchanged
customresourcedefinition.apiextensions.k8s.io/triggerauthentications.keda.sh unchanged
serviceaccount/keda-operator unchanged
clusterrole.rbac.authorization.k8s.io/keda-external-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/keda-operator configured
rolebinding.rbac.authorization.k8s.io/keda-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-hpa-controller-external-metrics unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-operator unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-system-auth-delegator unchanged
service/keda-metrics-apiserver unchanged
service/keda-operator unchanged
deployment.apps/keda-metrics-apiserver configured
deployment.apps/keda-operator c

In [85]:
#check for the keda components
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n keda"

NAME                                      READY   STATUS    RESTARTS        AGE
keda-metrics-apiserver-5b4c56c8c8-k7f2s   1/1     Running   4 (6d5h ago)    27d
keda-operator-598b95d667-xvd8b            1/1     Running   2191 (5d ago)   27d


In [86]:
#Create Scaler
!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl apply -f deploy-consumer-aeneas.yaml"


secret/rabbitmq-consumer-secret created
scaledobject.keda.sh/rabbitmq-aeneas created
triggerauthentication.keda.sh/rabbitmq-consumer-trigger created


In [87]:
#Create Scaler
#!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl apply -f deploy-consumer-getobject.yaml"


In [88]:
#Create Scaler
#!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl apply -f deploy-consumer-pubrabbitmq.yaml"


In [89]:
#Create Scaler
!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl apply -f deploy-consumer-tocloud.yaml"


secret/rabbitmq-consumer-secret unchanged
scaledobject.keda.sh/rabbitmq-tocloud created
triggerauthentication.keda.sh/rabbitmq-consumer-trigger unchanged


# Experiments 

In [135]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [136]:
scenario = "no_scaling"
user=10
iteration=0.05
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'
time_locust=120


In [137]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    client.remove_object(bucket_name, obj.object_name)

In [138]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv

In [139]:
!locust --headless -f testingrabbitlocust.py --host='127.0.0.1' --skip-log -t 120s  --users 10 --spawn-rate 0.05

In [133]:
time.sleep(100)

In [140]:
df= pd.read_csv('input_data.csv')
print(df)

                          intime
0   '2023-01-19 10:26:12.652810'
1   '2023-01-19 10:26:24.607499'
2   '2023-01-19 10:26:31.776312'
3   '2023-01-19 10:26:32.643660'
4   '2023-01-19 10:26:37.548220'
5   '2023-01-19 10:26:40.408382'
6   '2023-01-19 10:26:47.309624'
7   '2023-01-19 10:26:48.737809'
8   '2023-01-19 10:26:52.646094'
9   '2023-01-19 10:26:55.698129'
10  '2023-01-19 10:26:59.274543'
11  '2023-01-19 10:27:00.588515'
12  '2023-01-19 10:27:02.179599'
13  '2023-01-19 10:27:09.982444'
14  '2023-01-19 10:27:10.115100'
15  '2023-01-19 10:27:12.646901'
16  '2023-01-19 10:27:13.361787'
17  '2023-01-19 10:27:18.375010'
18  '2023-01-19 10:27:21.059475'
19  '2023-01-19 10:27:21.882914'
20  '2023-01-19 10:27:25.569839'
21  '2023-01-19 10:27:26.914038'
22  '2023-01-19 10:27:28.115189'
23  '2023-01-19 10:27:32.314760'
24  '2023-01-19 10:27:32.646688'
25  '2023-01-19 10:27:38.483069'
26  '2023-01-19 10:27:39.374371'
27  '2023-01-19 10:27:39.392417'
28  '2023-01-19 10:27:41.631926'
29  '2023-

In [141]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)

In [142]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime'],errors='coerce').dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'],errors='coerce')
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2023-01-19 10:26:12.652810,2023-01-19 10:26:17.056
1,2023-01-19 10:26:24.607499,2023-01-19 10:26:29.002
2,2023-01-19 10:26:31.776312,2023-01-19 10:26:37.038
3,2023-01-19 10:26:32.643660,2023-01-19 10:26:39.502
4,2023-01-19 10:26:37.548220,2023-01-19 10:26:44.608
5,2023-01-19 10:26:40.408382,2023-01-19 10:26:46.954
6,2023-01-19 10:26:47.309624,2023-01-19 10:26:52.644
7,2023-01-19 10:26:48.737809,2023-01-19 10:26:56.196
8,2023-01-19 10:26:52.646094,2023-01-19 10:26:59.593
9,2023-01-19 10:26:55.698129,2023-01-19 10:27:03.038


0 days 00:02:42.013190


In [143]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))

In [144]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn  -n openfaas -c gateway  | grep /function/aeneas | grep 200 | cut -c 80-83 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
cmd3="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/getobject | grep POST | cut -c 71-76  | tail -"+str(len(outtime))
cmd4="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/pubrabbitmq | grep POST | cut -c 73-78 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
#df['aeneas']=(pd.to_timedelta(df['aeneas'].astype(float), unit='s'))

df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
#df['tocloud']=(pd.to_timedelta(df['tocloud'].astype(float), unit='s'))

#df['getobject']=(get_function_execution_time(cmd3)).split("\n")[:-1]
df['pubrabbitmq']=((get_function_execution_time(cmd4)).split("\n")[:-1])
#df['pubrabbitmq']=(pd.to_timedelta(df['pubrabbitmq'].astype(float), unit='s'))

df

,intime,outtime,aeneas,tocloud,pubrabbitmq
0,2023-01-19 10:26:12.652810,2023-01-19 10:26:17.056,4.31,0.0357,0.0348
1,2023-01-19 10:26:24.607499,2023-01-19 10:26:29.002,4.30,0.0487,0.0474
2,2023-01-19 10:26:31.776312,2023-01-19 10:26:37.038,5.16,0.0364,0.0321
3,2023-01-19 10:26:32.643660,2023-01-19 10:26:39.502,6.77,0.0414,0.0357
4,2023-01-19 10:26:37.548220,2023-01-19 10:26:44.608,6.97,0.0760,0.0304
5,2023-01-19 10:26:40.408382,2023-01-19 10:26:46.954,6.43,0.0418,0.0306
6,2023-01-19 10:26:47.309624,2023-01-19 10:26:52.644,5.25,0.0388,0.0343
7,2023-01-19 10:26:48.737809,2023-01-19 10:26:56.196,7.39,0.0461,0.0955
8,2023-01-19 10:26:52.646094,2023-01-19 10:26:59.593,6.85,0.0393,0.0298
9,2023-01-19 10:26:55.698129,2023-01-19 10:27:03.038,7.24,0.0422,0.0385


In [145]:
#df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime'])
df['TPT']=df['TPT'].values.astype('datetime64[ns]')
df['TPT']= pd.to_datetime(df['TPT']).dt.strftime('%S.%f')
df['TPT']=df['TPT'].astype(float)
df['PT']= df['TPT'].round(decimals = 0)
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
#df['MQ time']= ((df['TPT']).astype(float) - df['FET'])
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
display(df)
df.to_csv(""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')

,intime,outtime,aeneas,tocloud,pubrabbitmq,TPT,PT,FET,scenario
0,2023-01-19 10:26:12.652810,2023-01-19 10:26:17.056,4.31,0.0357,0.0348,4.403190,4.0,4.3457,10_0.05_no_scaling
1,2023-01-19 10:26:24.607499,2023-01-19 10:26:29.002,4.30,0.0487,0.0474,4.394501,4.0,4.3487,10_0.05_no_scaling
2,2023-01-19 10:26:31.776312,2023-01-19 10:26:37.038,5.16,0.0364,0.0321,5.261688,5.0,5.1964,10_0.05_no_scaling
3,2023-01-19 10:26:32.643660,2023-01-19 10:26:39.502,6.77,0.0414,0.0357,6.858340,7.0,6.8114,10_0.05_no_scaling
4,2023-01-19 10:26:37.548220,2023-01-19 10:26:44.608,6.97,0.0760,0.0304,7.059780,7.0,7.0460,10_0.05_no_scaling
5,2023-01-19 10:26:40.408382,2023-01-19 10:26:46.954,6.43,0.0418,0.0306,6.545618,7.0,6.4718,10_0.05_no_scaling
6,2023-01-19 10:26:47.309624,2023-01-19 10:26:52.644,5.25,0.0388,0.0343,5.334376,5.0,5.2888,10_0.05_no_scaling
7,2023-01-19 10:26:48.737809,2023-01-19 10:26:56.196,7.39,0.0461,0.0955,7.458191,7.0,7.4361,10_0.05_no_scaling
8,2023-01-19 10:26:52.646094,2023-01-19 10:26:59.593,6.85,0.0393,0.0298,6.946906,7.0,6.8893,10_0.05_no_scaling
9,2023-01-19 10:26:55.698129,2023-01-19 10:27:03.038,7.24,0.0422,0.0385,7.339871,7.0,7.2822,10_0.05_no_scaling


In [146]:
pre_url = PROMETHEUS + '/api/v1/query_range?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time_interval = (time.mktime(timestamp.timetuple()))
user=df.shape[0]
end=(time.mktime(timestamp.timetuple()))
start=pd.to_datetime(df['intime'].iloc[0], format='%Y-%m-%d %H:%M:%S')
start=(time.mktime(start.timetuple()))
print(start,end)

1674123972.0 1674124134.0


In [147]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [148]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

162


In [149]:
import copy 
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    #data2=res.get('data').get('result')[0].get('values')
    data_dict={}
    metric_list = []
    # print(data['data']['result']['values'])
    # exit()
    for i in res['data']['result']:
        for j in i['values']:
            data_dict = copy.deepcopy(i['metric'])
            data_dict['time'] = j[0]
            data_dict['value'] = j[1]
            metric_list.append(data_dict)
    df_metric = pd.DataFrame(metric_list)
    return df_metric

In [150]:
user=df.shape[0]
metrics = {'workload_type':""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration),'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean()}

promo_metrics = pd.DataFrame(columns=['cpu_used','cpu_requested'])

In [151]:

expr_namespace_cpu_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['value']

promo_metrics['time']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['time']
expr_namespace_cpu_used = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)['value']

expr_namespace_pod_count = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['pod_count']=getdataprometheus(pre_url+expr_namespace_pod_count)['value']

expr_namespace_memory_used = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_used)['value']

expr_namespace_memory_requested = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_requested)['value']

promo_metrics['time']= pd.to_datetime(promo_metrics['time'],unit='s')
seconds = []
for i in range(promo_metrics.shape[0]): 
    seconds.append(i*10)
promo_metrics['seconds'] = seconds

throughput_qurty = 'sum (rate(gateway_function_invocation_total{code="200"}[10s]))'
promo_metrics['throughput']= getdataprometheus('http://172.17.141.197:31376/api/v1/query_range?query=sum (rate(gateway_function_invocation_total{code="200"}[20s]))&start='+str(start)+'&end='+str(end)+'&step=10s')['value']
promo_metrics.to_csv("keda_results/promo_metrics"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv',index=False)


# #expr_namespace_cpu_requested = 'sum(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace="openfaas-fn"})&time='+str(time_interval)
# expr_namespace_cpu_used='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)
# expr_namespace_cpu_requested='sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&time='+str(time_interval)
# metrics['CPU_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)

# expr_pod_cpu_rabbitmq_connector = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector)
# metrics['Memory_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_memory_rabbitmq_connector)

# expr_pod_cpu_rabbitmq_connector_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas", resource="cpu",pod=~"rabbitmq.*"})&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_requested']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector_requested)

# expr_pod_rabbitmq_function = 'sum(kube_pod_container_status_ready{namespace="openfaas",pod=~"rabbitmq.*"})&time='+str(time_interval)
# metrics['pod_count_rabbitmq']=getdataprometheus(pre_url+expr_pod_rabbitmq_function)

# expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})&time='+str(time_interval)
# expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"})&time='+str(time_interval)
# expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})&time='+str(time_interval)
# print(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
# metrics['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)

# metrics

In [152]:
df_metrics = pd.read_csv("overall_keda.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)
display(df_metrics)

/tmp/ipykernel_24585/4126905209.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(metrics, ignore_index=True)


,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,11.965319,143.776106,45,45_120_keda,11.506742
1,8.975417,128.887709,47,47_120_keda,8.582521
2,8.472999,123.443820,44,44_120_keda,8.069307
3,13.348710,141.320173,47,47_120_keda_0.05,12.668977
4,11.792392,132.404950,43,43_120_keda_all_0.05,9.096047
5,11.583683,140.159351,43,43_120_keda_all_polling_2_0.05,10.409893
6,16.302705,142.628210,43,43_120_keda_all_polling_2_0.05,15.636219
7,24.989442,172.641752,45,45_120_keda_all_iteration_2_0.05,24.074100
8,12.828964,141.403848,42,42_120_keda_all_iteration_2_0.05,12.298543
9,14.630457,147.714537,42,42_120_keda_keda_0.05,13.837902


In [153]:
df_metrics.to_csv("overall_keda.csv",index=False)
display(df_metrics)

,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,11.965319,143.776106,45,45_120_keda,11.506742
1,8.975417,128.887709,47,47_120_keda,8.582521
2,8.472999,123.443820,44,44_120_keda,8.069307
3,13.348710,141.320173,47,47_120_keda_0.05,12.668977
4,11.792392,132.404950,43,43_120_keda_all_0.05,9.096047
5,11.583683,140.159351,43,43_120_keda_all_polling_2_0.05,10.409893
6,16.302705,142.628210,43,43_120_keda_all_polling_2_0.05,15.636219
7,24.989442,172.641752,45,45_120_keda_all_iteration_2_0.05,24.074100
8,12.828964,141.403848,42,42_120_keda_all_iteration_2_0.05,12.298543
9,14.630457,147.714537,42,42_120_keda_keda_0.05,13.837902


In [154]:
df['PT'].describe()
df['PT'].quantile([0.0, .9])

0.0     4.0
0.9    46.4
Name: PT, dtype: float64

In [155]:
# Frequency
stats_df = df \
.groupby('PT') \
['PT'] \
.agg('count') \
.pipe(pd.DataFrame) \
.rename(columns = {'PT': 'frequency'})

# PDF
stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
stats_df['cdf'] = stats_df['pdf'].cumsum()
stats_df = stats_df.reset_index()
display(stats_df)
stats_df.to_csv("stats_df_"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
# from matplotlib import pyplot as plt

# plt.plot(stats_df['PT'],stats_df['cdf'],  linestyle='--', marker='o', color='b', label='k8s_k8s')
# #plt.plot(df['user'],df['CT'],  linestyle='--', marker='*', color='r', label='Communication Time')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('PT (Seconds)', fontsize=18)
# plt.ylabel('CDF', fontsize=16)


,PT,frequency,pdf,cdf
0,4.0,2,0.045455,0.045455
1,5.0,2,0.045455,0.090909
2,7.0,6,0.136364,0.227273
3,8.0,1,0.022727,0.250000
4,10.0,3,0.068182,0.318182
5,11.0,2,0.045455,0.363636
6,13.0,1,0.022727,0.386364
7,14.0,1,0.022727,0.409091
8,16.0,1,0.022727,0.431818
9,17.0,1,0.022727,0.454545


In [156]:
!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl delete -f deploy-consumer.yaml -n openfaas;sudo kubectl delete -f deploy-consumer-getobject.yaml -n openfaas-fn;sudo kubectl delete -f deploy-consumer-pubrabbitmq.yaml -n openfaas-fn;sudo kubectl delete -f deploy-consumer-tocloud.yaml -n openfaas-fn"


Error from server (NotFound): error when deleting "deploy-consumer.yaml": secrets "rabbitmq-consumer-secret" not found
Error from server (NotFound): error when deleting "deploy-consumer.yaml": scaledobjects.keda.sh "rabbitmq-connector" not found
Error from server (NotFound): error when deleting "deploy-consumer.yaml": triggerauthentications.keda.sh "rabbitmq-consumer-trigger" not found
Error from server (NotFound): error when deleting "deploy-consumer-getobject.yaml": secrets "rabbitmq-consumer-secret" not found
Error from server (NotFound): error when deleting "deploy-consumer-getobject.yaml": scaledobjects.keda.sh "rabbitmq-getobject" not found
Error from server (NotFound): error when deleting "deploy-consumer-getobject.yaml": triggerauthentications.keda.sh "rabbitmq-consumer-trigger" not found
Error from server (NotFound): error when deleting "deploy-consumer-pubrabbitmq.yaml": secrets "rabbitmq-consumer-secret" not found
Error from server (NotFound): error when deleting "deploy-con